# Analysis of JillyAtlanta Orders
***

In [1]:
import pandas as pd
import glob
from matplotlib import pyplot as plt

## Etsy

Get order data

In [12]:
dfe = pd.DataFrame()
fiels = glob.glob('./etsy/EtsySoldOrderItems/*')
for fiel in fiels:
    dfi = pd.read_csv(fiel, index_col=0)
    dfe = pd.concat([dfe, dfi])  
dfe.index = pd.to_datetime(dfe.index)
dfe.sort_index(inplace=True)
dfe.head()

,Item Name,Buyer,Quantity,Price,Coupon Code,Coupon Details,Discount Amount,Shipping Discount,Shipping,Sales Tax,...,Variations,Order Type,Listings Type,Payment Type,InPerson Discount,InPerson Location,VAT Paid by Buyer,SKU,Order Shipping,Order Sales Tax
Sale Date,,,,,,,,,,,,,,,,,,,,,
2014-06-16,Garden Skirt,Lauren Ward (LWard43),1,52.0,NaN,NaN,0.0,0.0,6.0,0.0,...,NaN,online,listing,online_cc,NaN,NaN,0.0,NaN,NaN,NaN
2014-08-30,"Reserved Listing: girl organic shorts, blue or...",masonjunk6772,1,40.0,NaN,NaN,0.0,0.0,6.0,0.0,...,Size:US 14,online,listing,online_cc,NaN,NaN,0.0,NaN,NaN,NaN
2014-08-30,Custom order--Size 12/14,masonjunk6772,1,52.0,NaN,NaN,0.0,0.0,2.0,0.0,...,Size:US 12/14,online,listing,online_cc,NaN,NaN,0.0,NaN,NaN,NaN
2014-08-31,"Reserved Listing: girls shorts, organic chamb...",masonjunk6772,1,40.0,NaN,NaN,0.0,0.0,6.0,0.0,...,Size:US 14,online,listing,online_cc,NaN,NaN,0.0,NaN,NaN,NaN
2014-09-02,"Reserved Listing: girl top, organic cotton, ye...",masonjunk6772,1,54.0,FREESHIPPING65,FREESHIPPING65 - free shipping!,0.0,8.0,0.0,0.0,...,Size:US 14,online,listing,online_cc,NaN,NaN,0.0,NaN,NaN,NaN


Cull columns

In [3]:
all_columns = list(dfe)
# ['Item Name', 'Buyer', 'Quantity', 'Price', 'Coupon Code', 'Coupon Details', 'Discount Amount', 'Shipping Discount', 'Shipping', 'Sales Tax', 'Item Total', 'Currency', 'Transaction ID', 'Listing ID', 'Date Paid', 'Date Shipped', 'Ship Name', 'Ship Address1', 'Ship Address2', 'Ship City', 'Ship State', 'Ship Zipcode', 'Ship Country', 'Order ID', 'Variations', 'Order Type', 'Listings Type', 'Payment Type', 'InPerson Discount', 'InPerson Location', 'VAT Paid by Buyer', 'SKU', 'Order Shipping', 'Order Sales Tax']
columns_Iwant = [
    'Item Name', 
    'Quantity', 
    'Price',
    ]
dfe = dfe.loc[:,columns_Iwant]
dfe.columns = ['item_name', 'quantity', 'price']

Get list of all PDF listings names

In [4]:
listings = pd.read_csv('listings_names.csv')
pdf_listings = listings[listings['PDF'] == 1]
pdfs = list(pdf_listings.item_title.values)
pdfs_clean = [x.strip(" '") for x in pdfs]

Make a new column indicating if a sale is a PDF (True) or not (False)

In [5]:
def is_pdf(item_name):
    return item_name in pdfs_clean

dfe['pdf'] = dfe['item_name'].apply(is_pdf)

Cull all non-pdf sales from dataframe

In [8]:
dfe = dfe[dfe['pdf'] == True]

## Shopify

These data not available in standard reports.  Customer service is working on getting it to me.

In [7]:
dfs = pd.read_csv('./shopify/sales.csv')
dfs = dfs[dfs['total_sales'] != 0]
dfs.head()

,day,product_title,orders,gross_sales,discounts,returns,net_sales,total_sales
91,2020-04-04,Agatha Top + Dress,1,9.0,0.00,0.00,9.00,9.63
103,2020-04-16,NaN,0,0.0,0.00,12.84,12.84,12.84
105,2020-04-17,Lola Blouse,5,45.0,-15.75,0.00,29.25,29.25
106,2020-04-17,Mona Shorts,2,18.0,-6.30,0.00,11.70,11.70
107,2020-04-17,Hastings Vest,1,9.0,-3.15,0.00,5.85,5.85
